In [19]:
%pylab inline
import os
import pandas as pd
from rdkit import Chem
import numpy as np
from rdkit.Chem import rdMolAlign, rdMolTransforms, rdMolDescriptors, rdDistGeom, ChemicalForceFields

Populating the interactive namespace from numpy and matplotlib


In [20]:
def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 


def getRMS(mol, CS_result):
    rmsd_list=[]
    for i in CS_result:
        b = rdMolAlign.AlignMol(mol,i)
        if b > 0.00001: 
            rmsd_list.append(b)
    return np.mean(rmsd_list)

#mol = a given molecular structure 
#CS_result = list of molecular structures in the mol file 
#this function compare the similarity of mol with the structures in CS_result with AlignMol and give an 
#averged AlignMol value


def getRMSlen(mol, CS_result, no):
    rmsd_list2=[]
    for i in CS_result:
        a = rdMolAlign.AlignMol(mol,i)
        if a < no: 
            rmsd_list2.append(a)
    return len(rmsd_list2)

#mol = a given molecular structure 
#CS_result = list of molecular structures in the mol file 
#no = a value; if AlignMol of mol and the structure from CS_search is less than no, append the number into a list
#count the number of item in the list

def getkeyindex(filename, no, percentage):
    
    suppl = Chem.SDMolSupplier(filename)
    CS_result= [mol for mol in suppl]
    
    #test 1: by average number
    
    avg_list=[]
    for u in CS_result:
        avg_list.append(getRMS(u, CS_result))
    
    #test 2: number of structures with RMS less than 1.0 
    
    len_list=[]
    for q in CS_result:
        len_list.append(getRMSlen(q, CS_result, no))
    
    top_percent=np.floor(percentage*len(CS_result))


    avg_top_idx = np.argsort(avg_list)[-int(top_percent):]
    avg_top_values = [avg_list[i] for i in avg_top_idx]

    len_top_idx = np.argsort(len_list)[:len_list.count(1)]
    len_top_values = [len_list[i] for i in len_top_idx]

    resulting_list=list(len_top_idx) + list(set(list(avg_top_idx)) - set(list(len_top_idx)))
    #result_true_indx=[r+1 for r in resulting_list]
    
    return resulting_list


#right spacing for atomic coordinates in gjf fils
def right_spacing(coor_list):
    coor_list_space=[]
    for cline in coor_list:
        split_coor=cline.split()
        new_coor=''
        space1=18
        space2=4
        space3=4
        if len(split_coor[0]) == 2:
            space1=17
        for o in split_coor:  
            if '-' in o:
                index_list=[]
                index_list.append(split_coor.index(o))
                if 1 in index_list:
                    space1=17
                elif 2 in index_list:
                    space2=3
                elif 3 in index_list:
                    space3=3
        new_coor=str(split_coor[0])+space1*' '+str(split_coor[1])+'000'+space2*' '+str(split_coor[2])+'000'+space3*' '+str(split_coor[3])+'000\n'
        coor_list_space.append(new_coor)
    return coor_list_space

#convert to gjf file 
def copy_structure(filename_xyz, filename_sdf, keywords,space, keynumber, no=1.2, percentage=0.1):
    
    xyz_output=[]

    with open(filename_xyz) as output:
        for line in output:
            xyz_output+= [line]

    Eopt_index_list =[i for i,d in enumerate(xyz_output) if d=='Molecule Name\n']


    division1=[]
    for j in range(0,len(Eopt_index_list)):
        try:
            a=xyz_output[Eopt_index_list[j]+1: Eopt_index_list[j+1]-1]
            division1.append(a)
        except IndexError:
            b=xyz_output[Eopt_index_list[j]+1:]
            division1.append(b)

    division2=[]
    division3=[]
    for s in division1:
        division2.append(right_spacing(s))

    for structure in division2: 
        coor_content=''
        for individual_structure in structure:
            coor_content+=str(individual_structure)
        content=keywords+coor_content+space
        division3.append(content)

      
    #intergrating RDkit Alignment 
    
    resulting_list = getkeyindex(filename_sdf, no, percentage)
    result_true_indx=[r+1 for r in resulting_list]
    
    
    select_pre=list(range(0,len(division3),keynumber)) + resulting_list
    select= Remove(select_pre)
    select1=[y+1 for y in select]
    
    print('Additional structures: '+str(result_true_indx))
    print('All the selected structures: '+ str(select1))
    print('Total no. of structures: '+ str(len(select1)))
    print()
    
    #end of intergrating RDkit Alignment 
    
    division4=[]
    for o in select:
        division4.append(division3[o])
    
    new_filename= filename_xyz[:-4]
    
    
    filename_list2=[]
    for num2 in select1:
        filename_list2.append(new_filename+'_'+str(num2)+'.gjf')
    
    for c,d in zip(filename_list2,division4):
        print(c+'\n',d)
        file = open(c, "w") 
        file.write(d) 
        file.close() 
    
    return 

In [21]:
keywords='''%nprocshared=8
%mem=1GB
# opt freq b3lyp/6-31g(d) empiricaldispersion=gd3 int=ultrafine

Title Card Required

0 2
''' 
space='''
'''

In [22]:
copy_structure('xanthone_d5mC.xyz','xanthone_d5mC.sdf', keywords,space,4,1,0.05)

Additional structures: [39, 2, 3, 17, 10, 11, 16, 9]
All the selected structures: [1, 5, 9, 13, 17, 21, 25, 29, 33, 37, 39, 2, 3, 10, 11, 16]
Total no. of structures: 16

xanthone_d5mC_1.gjf
 %nprocshared=8
%mem=1GB
# opt freq b3lyp/6-31g(d) empiricaldispersion=gd3 int=ultrafine

Title Card Required

0 2
C                  2.66680000    0.58280000    3.63060000
C                  2.60110000    1.71670000    2.81750000
C                  2.45750000   -0.83340000    1.68350000
C                  2.59500000   -0.69100000    3.06450000
C                  2.18030000    3.85180000   -1.35660000
H                  2.25590000    4.79780000   -0.82640000
C                  2.01950000    3.84470000   -2.74390000
C                  1.91780000    2.63440000   -3.43020000
C                  1.97620000    1.42900000   -2.73050000
H                  2.77300000    0.69300000    4.70750000
H                  2.65740000    2.70650000    3.26380000
H                  2.40020000   -1.82630000    1.2432000